# 5강) BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

In [1]:
import torch
torch.set_printoptions(threshold=10_000)

In [2]:
# !pip install datasets
# !pip install transformers

## 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [3]:
from datasets import load_dataset, load_from_disk

# dataset = load_dataset("squad_kor_v1")
dataset = load_from_disk('../../data/train_dataset')

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [4]:
from transformers import AutoTokenizer
import numpy as np

# model_checkpoint = "bert-base-multilingual-cased"
# model_checkpoint = "klue/roberta-base"
model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [5]:
tokenizer

PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 미국 상의원 또는 미국 상원 ( United States Senate ) 은 양원제인 미국 의회의 상원이다. [UNK] n [UNK] n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1 / 3씩 상원의원을 새로 선출하여 연방에 보낸다. [UNK] n [UNK] n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할 ( 하원의 법안을 거부할 권한 등 ) 을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션 ( 공공건강보험기관 ) 의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다. 날짜 = 2017 - 02 - 05 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [7]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [8]:
# Use subset (128 example) of original training dataset 
# sample_idx = np.random.choice(range(len(dataset['train'])), 128)
# training_dataset = dataset['train'][sample_idx]

training_dataset = dataset['train']

In [9]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

# q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt', return_token_type_ids=False)
# p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt', return_token_type_ids=False)
q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')


In [10]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'],)

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [11]:
from transformers import RobertaPreTrainedModel, RobertaModel

In [12]:
# class BertEncoder(RobertaPreTrainedModel):
class BertEncoder(BertPreTrainedModel):  
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    # self.bert = RobertaModel(config)
    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids #roberta시 주석
                          )
      
      pooled_output = outputs[1]

      return pooled_output


In [13]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

# p_encoder = BertEncoder.from_pretrained('../encoders/p_encoder')
# q_encoder = BertEncoder.from_pretrained('../encoders/q_encoder')

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.de

In [14]:
# p_encoder

In [15]:
# for i in iter(p_encoder.named_parameters()):
#     print(i)

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용) 


In [21]:
def train(args, dataset, p_model, q_model):
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
    {"params": [p for n, p in p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
    {"params": [p for n, p in p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    {"params": [p for n, p in q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
    {"params": [p for n, p in q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
  ]

  optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=args.learning_rate,
    eps=args.adam_epsilon
  )
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

# tt
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2] # roberta시 주석
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5] # roberta시 주석
                  }
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)
      
      # print('q_outputs: ',q_outputs)
      # print('p_outputs: ',p_outputs)
      # print('sim_scores: ',sim_scores)
        
      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)
      
      

      loss = F.nll_loss(sim_scores, targets)
      # print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model




In [22]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01
)


In [23]:
from tqdm import tqdm

In [24]:
p_encoder, q_encoder = tqdm(train(args, train_dataset, p_encoder, q_encoder))

100%|██████████| 2/2 [00:00<00:00, 6887.20it/s]


In [25]:
p_encoder.save_pretrained('../encoders/p_encoder')
q_encoder.save_pretrained('../encoders/q_encoder')

In [7]:
p_encoder = BertEncoder.from_pretrained('../encoders/p_encoder_neg').to('cuda')
q_encoder = BertEncoder.from_pretrained('../encoders/q_encoder_neg').to('cuda')

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [26]:

valid_corpus = list(set([example['context'] for example in dataset['validation']]))#[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

미국 남부 지방에서 급격하게 영국산 제품이 많이 팔린 계기가 된 조치는?
통상금지법(Embargo Act of 1807)은 1807년에 미국 의회에서 통과된 법률이며 , 나폴레옹 전쟁을 치루고 있는 영국과 프랑스에 대항하여 미국의 무역을 금지한 것이었다\n\n이 통상금지(이하, 엠바고)는 미국의 상인과 화물선이 유럽의 해군에게 금수품으로 나포하면서, 미국의 중립 위반에 대한 보복 조치를 취하게 되자 이 법안을 시행하게 되었다. 영국 해군은 특히 수천 명의 미국 선원들을 자신들의 함대에 복역하게 하는 강제 징발 조치를 감행하였다. 유럽에 대한 통제권을 위해 필사적으로 다투었던 영국과 프랑스는 우발적인 사고이고, 그들의 생존을 위해 필수적인 조치였다고 합리화했다. 미국인들은 체사피크 레오퍼드 사건을 중립을 지키는 미국의 조치에 대한 영국의 어처구니 없는 위반 사례라고 보았다 외교적 모욕이라고 이해를 하고, 유럽 열강에 의해 이러한 작전들을 지지하는 보증되지 않은 공식적인 명령서는 미국의 선전포고에 대한 근거로서 널리 인정되고 있다. 노한 군중들은 냉소적으로 ‘엠바고’(Embargo)라는 글자를 거꾸로 ‘나 좀 잡아줘’(O Grab Me)라고 바꿔 불렀다. \n\n토머스 제퍼슨 대통령은 이러한 분노한 대중들의 적의를 등에 업고, 보복에 대한 대중적 지지에 힘을 싣고 있었다. 그는 의회에 군사적으로 물리적 조치보다는 경제 전쟁으로 응전해야 한다고 권했다. 그리하여 통상금지법이 1807년 12월 22일 서명되었다. 이 급작스런 조치에 대한 기대 효과(교전국 사이의 경제적 곤란 )는 영국과 프랑스를 힘들게 하여, 그들로 하여금 미국 상선을 방해를 하지못하도록 강제하고, 미국의 중립을 존중하며, 강제 징발 정책을 끝내는 것이었다. 이 엠바고는 강압적인 조치로서는 그다지 효과를 보지 못한 것으로 밝혀졌으며, 외교적으로도, 경제적으로 모두 실패한 조치였다. 이 법이 막상 시행되고 나서, 미국 경제와 미국 국민들에게 파괴적인 부담을 주었다. \n\n미국 상인들에 의한 바다와 내

앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [27]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [28]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  # q_seqs_val = tokenizer(['미국의 대통령은 누구인가?'], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([235, 768]) torch.Size([1, 768])


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [29]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
# print(dot_prod_scores)
# print(rank)

torch.Size([1, 235])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [30]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 미국 남부 지방에서 급격하게 영국산 제품이 많이 팔린 계기가 된 조치는? 

[Ground truth passage]
통상금지법(Embargo Act of 1807)은 1807년에 미국 의회에서 통과된 법률이며 , 나폴레옹 전쟁을 치루고 있는 영국과 프랑스에 대항하여 미국의 무역을 금지한 것이었다\n\n이 통상금지(이하, 엠바고)는 미국의 상인과 화물선이 유럽의 해군에게 금수품으로 나포하면서, 미국의 중립 위반에 대한 보복 조치를 취하게 되자 이 법안을 시행하게 되었다. 영국 해군은 특히 수천 명의 미국 선원들을 자신들의 함대에 복역하게 하는 강제 징발 조치를 감행하였다. 유럽에 대한 통제권을 위해 필사적으로 다투었던 영국과 프랑스는 우발적인 사고이고, 그들의 생존을 위해 필수적인 조치였다고 합리화했다. 미국인들은 체사피크 레오퍼드 사건을 중립을 지키는 미국의 조치에 대한 영국의 어처구니 없는 위반 사례라고 보았다 외교적 모욕이라고 이해를 하고, 유럽 열강에 의해 이러한 작전들을 지지하는 보증되지 않은 공식적인 명령서는 미국의 선전포고에 대한 근거로서 널리 인정되고 있다. 노한 군중들은 냉소적으로 ‘엠바고’(Embargo)라는 글자를 거꾸로 ‘나 좀 잡아줘’(O Grab Me)라고 바꿔 불렀다. \n\n토머스 제퍼슨 대통령은 이러한 분노한 대중들의 적의를 등에 업고, 보복에 대한 대중적 지지에 힘을 싣고 있었다. 그는 의회에 군사적으로 물리적 조치보다는 경제 전쟁으로 응전해야 한다고 권했다. 그리하여 통상금지법이 1807년 12월 22일 서명되었다. 이 급작스런 조치에 대한 기대 효과(교전국 사이의 경제적 곤란 )는 영국과 프랑스를 힘들게 하여, 그들로 하여금 미국 상선을 방해를 하지못하도록 강제하고, 미국의 중립을 존중하며, 강제 징발 정책을 끝내는 것이었다. 이 엠바고는 강압적인 조치로서는 그다지 효과를 보지 못한 것으로 밝혀졌으며, 외교적으로도, 경제적으로 모두 실패한 조치였다. 이 법이 막상 시행되고 나서, 미국 경제와 미국